# Making firmware with hls4ml

This is the fun part! Now we'll convert our Keras model into highly parallel FPGA firmware with hls4ml, using only a few lines of code. This is the basic flow:

<img src="images/hls4ml_conifer.png" alt="hls4ml" width="1000" img align="center"/>

So what hlsm4l does is that it takes your TF/Keras/ONNX/Pytorch model and converts it into C++ code that a high-level-synthesis (HLS) tool can read. It automatically adds pipelining stages to your hardware depending on the target FPGA  and clock period constraints (usually at LHC we're on a 2 or 5 nanosecond clock).

After converting your model with hls, you'll see that a lot of C++ code gets generated, which is then passed to Vivado HLS for compilation.

Let's make bitfiles both of the large and the compressed model:

In [4]:
import hls4ml
import util
import h5py
import numpy as np
import tensorflow as tf

from qkeras.utils import _add_supported_quantized_objects
co = {}; _add_supported_quantized_objects(co)
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = []  
hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

with h5py.File('Ato4l_dataset.h5', 'r') as file:
    signal_test_data = np.array(file['Data'])

# First the baseline:
autoencoder = tf.keras.models.load_model('baseline_ae.h5')

config = hls4ml.utils.config_from_keras_model(autoencoder, granularity='name')
config['Model']['Strategy'] = 'Resource'

for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 64 #Use the same resources multiple times. This is neccessary in this case because the FPGA is small.
hls_model = hls4ml.converters.convert_from_keras_model(autoencoder,
                                                         hls_config=config,
                                                         backend='VivadoAccelerator', #You need this backend to generate firmware for Zynq
                                                         output_dir='/mnt/data/thaarres/baseline_ae_pynq',
                                                         board='pynq-z2') # This is our FPGA!
                                                   
hls_model.compile()

y_hls4ml = hls_model.predict(np.ascontiguousarray(signal_test_data))
hls_model.build(csim=False, synth=True, export=True)
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(hls_model)

# Package the model and some test data to be moved over to the Pynq
util.package(hls_model, signal_test_data, y_hls4ml)



Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: batch_normalization, layer type: BatchNormalization
Layer name: dense, layer type: Dense
  -> Activation (linear), layer name: dense
Layer name: batch_normalization_1, layer type: BatchNormalization
Layer name: leaky_re_lu, layer type: LeakyReLU
Layer name: dense_1, layer type: Dense
  -> Activation (linear), layer name: dense_1
Layer name: batch_normalization_2, layer type: BatchNormalization
Layer name: leaky_re_lu_1, layer type: LeakyReLU
Layer name: dense_2, layer type: Dense
  -> Activation (linear), layer name: dense_2
Layer name: dense_3, layer type: Dense
  -> Activation (linear), layer name: dense_3
Layer name: batch_normalization_3, layer type: BatchNormalization
Layer name: leaky_re_lu_2, layer type: LeakyReLU
Layer name: dense_4, layer type: Dense
  -> Activation (linear), layer name: dense_4
Layer name: batch_normalization_4, layer type: BatchNormalization
Layer name: leaky_re_lu_3, layer type:

INFO: [HLS 200-10] Analyzing design file 'firmware/myproject_axi.cpp' ... 
INFO: [HLS 200-111] Finished Linking Time (s): cpu = 00:00:27 ; elapsed = 00:00:27 . Memory (MB): peak = 1764.770 ; gain = 1227.961 ; free physical = 13046 ; free virtual = 49143
INFO: [HLS 200-111] Finished Checking Pragmas Time (s): cpu = 00:00:27 ; elapsed = 00:00:27 . Memory (MB): peak = 1764.770 ; gain = 1227.961 ; free physical = 13038 ; free virtual = 49135
INFO: [HLS 200-10] Starting code transformations ...
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:158) in function 'void nnet::dense_resource_rf_gt_nin_rem0<ap_fixed<16, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config13>(FORWARD_REFERENCE*, FORWARD_REFERENCE*, FORWARD_REFERENCE::weight_t*, FORWARD_REFERENCE::bias_t*)' completely with a factor of 1.
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:158) in function 'void nnet::den

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'InitAccum' (firmware/nnet_utils/nnet_dense_resource.h:58) in function 'nnet::dense_resource<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config21>' completely with a factor of 57.
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:73) in function 'nnet::dense_resource<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config21>' completely with a factor of 57.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_resource.h:97) in function 'nnet::dense_resource<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config21>' completely with a factor of 57.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_activation.h:516) in function 'nnet::leaky_relu<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_m

INFO: [XFORM 203-101] Partitioning array 'out_local'  in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer2_out.V' (firmware/myproject.cpp:76) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer3_out.V' (firmware/myproject.cpp:80) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer4_out.V' (firmware/myproject.cpp:84) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer5_out.V' (firmware/myproject.cpp:88) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer6_out.V' (firmware/myproject.cpp:92) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer7_out.V' (firmware/myproject.cpp:96) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer8_out.V' (firmware/myproject.cpp:100) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer9_out.V' (firmware/myproject.cpp:104) in dimension 1 completely.
INFO: [XFORM 203-101] Partiti

...

Phase 2.2.1 Physical Synthesis In Placer | Checksum: 1fab4ccdc

Time (s): cpu = 00:02:00 ; elapsed = 00:01:34 . Memory (MB): peak = 3649.422 ; gain = 0.000 ; free physical = 1862 ; free virtual = 38655
Phase 2.2 Global Placement Core | Checksum: 1044188fe

Time (s): cpu = 00:05:22 ; elapsed = 00:03:20 . Memory (MB): peak = 3649.422 ; gain = 0.000 ; free physical = 2844 ; free virtual = 39639
Phase 2 Global Placement | Checksum: 1044188fe

Time (s): cpu = 00:05:22 ; elapsed = 00:03:20 . Memory (MB): peak = 3649.422 ; gain = 0.000 ; free physical = 2877 ; free virtual = 39672

Phase 3 Detail Placement

Phase 3.1 Commit Multi Column Macros
Phase 3.1 Commit Multi Column Macros | Checksum: 178e90821

Time (s): cpu = 00:05:28 ; elapsed = 00:03:24 . Memory (MB): peak = 3649.422 ; gain = 0.000 ; free physical = 4459 ; free virtual = 41253

Phase 3.2 Commit Most Macros & LUTRAMs
Phase 3.2 Commit Most Macros & LUTRAMs | Checksum: 17ce0bf11

Time (s): cpu = 00:05:38 ; elapsed = 00:03:32 . Memory 

INFO: [Common 17-1381] The checkpoint '/mnt/data/thaarres/baseline_ae_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_physopt.dcp' has been generated.
write_checkpoint: Time (s): cpu = 00:00:17 ; elapsed = 00:00:11 . Memory (MB): peak = 3649.422 ; gain = 0.000 ; free physical = 4707 ; free virtual = 41559
Command: route_design
Attempting to get a license for feature 'Implementation' and/or device 'xc7z020'
INFO: [Common 17-349] Got license for feature 'Implementation' and/or device 'xc7z020'
Running DRC as a precondition to command route_design
INFO: [DRC 23-27] Running DRC with 6 threads
INFO: [Vivado_Tcl 4-198] DRC finished with 0 Errors
INFO: [Vivado_Tcl 4-199] Please refer to the DRC report (report_drc) for more information.


Starting Routing Task
INFO: [Route 35-254] Multithreading enabled for route_design using a maximum of 6 CPUs
Checksum: PlaceDB: 72b0705c ConstDB: 0 ShapeSum: ad0895d5 RouteDB: 0

Phase 1 Build RT Design
Phase 1 Build RT Design | Check

Phase 9 Depositing Routes | Checksum: eeee39fc

Time (s): cpu = 00:03:36 ; elapsed = 00:02:57 . Memory (MB): peak = 3809.809 ; gain = 160.387 ; free physical = 2950 ; free virtual = 39806

Phase 10 Post Router Timing

Phase 10.1 Update Timing
Phase 10.1 Update Timing | Checksum: 1ce5e3dec

Time (s): cpu = 00:03:42 ; elapsed = 00:03:01 . Memory (MB): peak = 3809.809 ; gain = 160.387 ; free physical = 2768 ; free virtual = 39624
INFO: [Route 35-57] Estimated Timing Summary | WNS=-0.102 | TNS=-0.102 | WHS=0.050  | THS=0.000  |

Resolution: For a complete and accurate timing signoff, report_timing_summary must be run after route_design. Alternatively, route_design can be run with the -timing_summary option to enable a complete timing signoff at the end of route_design.
Phase 10 Post Router Timing | Checksum: 1ce5e3dec

Time (s): cpu = 00:03:42 ; elapsed = 00:03:01 . Memory (MB): peak = 3809.809 ; gain = 160.387 ; free physical = 2886 ; free virtual = 39742
INFO: [Route 35-16] Router Comple

INFO: [DRC 23-27] Running DRC with 6 threads
INFO: [Vivado 12-3199] DRC finished with 0 Errors, 16 Warnings
INFO: [Vivado 12-3200] Please refer to the DRC report (report_drc) for more information.


INFO: [Designutils 20-2272] Running write_bitstream with 6 threads.
Loading data files...
Loading site data...
Loading route data...
Processing options...
Creating bitmap...
Creating bitstream...
Writing bitstream ./design_1_wrapper.bit...
INFO: [Vivado 12-1842] Bitgen Completed Successfully.
INFO: [Common 17-83] Releasing license: Implementation
145 Infos, 21 Warnings, 1 Critical Warnings and 0 Errors encountered.
write_bitstream completed successfully
write_bitstream: Time (s): cpu = 00:00:36 ; elapsed = 00:00:38 . Memory (MB): peak = 4249.863 ; gain = 185.039 ; free physical = 2743 ; free virtual = 39151
INFO: [Common 17-206] Exiting Vivado at Thu Dec  2 10:02:19 2021...
[Thu Dec  2 10:02:25 2021] impl_1 finished
wait_on_run: Time (s): cpu = 00:09:57 ; elapsed = 00:19:19 . Memory (MB): peak = 2467.430 ; gain = 0.000 ; free physical = 5115 ; free virtual = 41525
# open_run impl_1
INFO: [Device 21-403] Loading part xc7z020clg400-1
Netlist sorting complete. Time (s): cpu = 00:00:00.41 

In [7]:
# Then the compressed model:
from qkeras.utils import _add_supported_quantized_objects
co = {}
_add_supported_quantized_objects(co)

q_autoencoder = tf.keras.models.load_model('compressed_ae.h5', custom_objects=co)

config = hls4ml.utils.config_from_keras_model(q_autoencoder, granularity='name')
config['Model']['Strategy'] = 'Resource'
for layer in config['LayerName'].keys():
    config['LayerName'][layer]['ReuseFactor'] = 64
q_hls_model = hls4ml.converters.convert_from_keras_model(q_autoencoder,
                                                         hls_config=config,
                                                         backend='VivadoAccelerator',
                                                         output_dir='/mnt/data/thaarres/qkeras_ae_pynq',
                                                         board='pynq-z2')                                                
q_hls_model.compile()

y_q_hls4ml = q_hls_model.predict(np.ascontiguousarray(signal_test_data))
q_hls_model.build(csim=False, synth=True, export=True)
hls4ml.templates.VivadoAcceleratorBackend.make_bitfile(q_hls_model)
util.package(q_hls_model, signal_test_data, y_q_hls4ml)

Interpreting Model
Topology:
Layer name: input_2, layer type: Input
Layer name: batch_normalization_5, layer type: BatchNormalization
Layer name: q_dense, layer type: QDense
Layer name: batch_normalization_6, layer type: BatchNormalization
Layer name: q_activation, layer type: QActivation
Layer name: q_dense_1, layer type: QDense
Layer name: batch_normalization_7, layer type: BatchNormalization
Layer name: q_activation_1, layer type: QActivation
Layer name: q_dense_2, layer type: QDense
Layer name: q_dense_3, layer type: QDense
Layer name: batch_normalization_8, layer type: BatchNormalization
Layer name: q_activation_2, layer type: QActivation
Layer name: q_dense_4, layer type: QDense
Layer name: batch_normalization_9, layer type: BatchNormalization
Layer name: q_activation_3, layer type: QActivation
Layer name: q_dense_5, layer type: QDense
Interpreting Model
Topology:
Layer name: input_2, layer type: InputLayer, input shapes: [[None, 57]], output shape: [None, 57]
Layer name: batch_n

INFO: [HLS 200-10] Analyzing design file 'firmware/myproject_axi.cpp' ... 
INFO: [HLS 200-111] Finished Linking Time (s): cpu = 00:00:26 ; elapsed = 00:00:27 . Memory (MB): peak = 1764.770 ; gain = 1227.961 ; free physical = 12601 ; free virtual = 49111
INFO: [HLS 200-111] Finished Checking Pragmas Time (s): cpu = 00:00:26 ; elapsed = 00:00:27 . Memory (MB): peak = 1764.770 ; gain = 1227.961 ; free physical = 12601 ; free virtual = 49111
INFO: [HLS 200-10] Starting code transformations ...
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:158) in function 'void nnet::dense_resource_rf_gt_nin_rem0<ap_fixed<16, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config13>(FORWARD_REFERENCE*, FORWARD_REFERENCE*, FORWARD_REFERENCE::weight_t*, FORWARD_REFERENCE::bias_t*)' completely with a factor of 1.
INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:158) in function 'void nnet::den

INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::mantissa' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:15) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::expv' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:18) automatically.
INFO: [XFORM 203-602] Inlining function 'fp_struct<double>::__signbit' into 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/FloatingPoint/hls_case_IEEE754.h:59) automatically.
INFO: [XFORM 203-602] Inlining function 'generic_cast_IEEE754<int, (ap_q_mode)6, double>' into 'generic_cast_IEEE754<int, double>' (/wrk/2020.1/continuous/2020_05_27_2902540/src/products/hls/hls_lib/hlsmath/include/Float

INFO: [HLS 200-489] Unrolling loop 'MultLoop' (firmware/nnet_utils/nnet_dense_resource.h:73) in function 'nnet::dense_resource<ap_ufixed<8, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config21>' completely with a factor of 57.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_dense_resource.h:97) in function 'nnet::dense_resource<ap_ufixed<8, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, config21>' completely with a factor of 57.
INFO: [HLS 200-489] Unrolling loop 'Loop-1' (firmware/nnet_utils/nnet_activation.h:76) in function 'nnet::relu<ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_ufixed<8, 0, (ap_q_mode)0, (ap_o_mode)0, 0>, relu_config20>' completely with a factor of 32.
INFO: [HLS 200-489] Unrolling loop 'Result' (firmware/nnet_utils/nnet_batchnorm.h:83) in function 'nnet::normalize<ap_fixed<16, 6, (ap_q_mode)0, (ap_o_mode)0, 0>, ap_fixed<16, 6, (ap_q_mode)5, (ap_o_mode)3, 0>, conf

INFO: [XFORM 203-101] Partitioning array 'out_local'  in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer2_out.V' (firmware/myproject.cpp:76) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer3_out.V' (firmware/myproject.cpp:80) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer4_out.V' (firmware/myproject.cpp:84) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer5_out.V' (firmware/myproject.cpp:88) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer6_out.V' (firmware/myproject.cpp:92) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer7_out.V' (firmware/myproject.cpp:96) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer8_out.V' (firmware/myproject.cpp:100) in dimension 1 completely.
INFO: [XFORM 203-101] Partitioning array 'layer9_out.V' (firmware/myproject.cpp:104) in dimension 1 completely.
INFO: [XFORM 203-101] Partiti

...

INFO: [Common 17-1381] The checkpoint '/mnt/data/thaarres/qkeras_ae_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_opt.dcp' has been generated.
write_checkpoint: Time (s): cpu = 00:00:12 ; elapsed = 00:00:09 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9775 ; free virtual = 46677
INFO: [runtcl-4] Executing : report_drc -file design_1_wrapper_drc_opted.rpt -pb design_1_wrapper_drc_opted.pb -rpx design_1_wrapper_drc_opted.rpx
Command: report_drc -file design_1_wrapper_drc_opted.rpt -pb design_1_wrapper_drc_opted.pb -rpx design_1_wrapper_drc_opted.rpx
INFO: [IP_Flow 19-1839] IP Catalog is up to date.
INFO: [DRC 23-27] Running DRC with 6 threads
INFO: [Coretcl 2-168] The results of DRC are in file /mnt/data/thaarres/qkeras_ae_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_drc_opted.rpt.
report_drc completed successfully
Command: place_design
Attempting to get a license for feature 'Implementation' and/or device 'xc7z020

Phase 2.2 Global Placement Core | Checksum: 1307cda73

Time (s): cpu = 00:02:18 ; elapsed = 00:00:58 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9248 ; free virtual = 46151
Phase 2 Global Placement | Checksum: 1307cda73

Time (s): cpu = 00:02:19 ; elapsed = 00:00:58 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9453 ; free virtual = 46355

Phase 3 Detail Placement

Phase 3.1 Commit Multi Column Macros
Phase 3.1 Commit Multi Column Macros | Checksum: 125812e6e

Time (s): cpu = 00:02:24 ; elapsed = 00:01:00 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9210 ; free virtual = 46112

Phase 3.2 Commit Most Macros & LUTRAMs
Phase 3.2 Commit Most Macros & LUTRAMs | Checksum: 19aa3f52e

Time (s): cpu = 00:02:33 ; elapsed = 00:01:04 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9305 ; free virtual = 46207

Phase 3.3 Area Swap Optimization
Phase 3.3 Area Swap Optimization | Checksum: 1e8bb91d5

Time (s): cpu = 00:02:34 ; el

Writing XDEF routing.
Writing XDEF routing logical nets.
Writing XDEF routing special nets.
Write XDEF Complete: Time (s): cpu = 00:00:06 ; elapsed = 00:00:03 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9703 ; free virtual = 46737
INFO: [Common 17-1381] The checkpoint '/mnt/data/thaarres/qkeras_ae_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_physopt.dcp' has been generated.
write_checkpoint: Time (s): cpu = 00:00:15 ; elapsed = 00:00:10 . Memory (MB): peak = 3578.852 ; gain = 0.000 ; free physical = 9800 ; free virtual = 46754
Command: route_design
Attempting to get a license for feature 'Implementation' and/or device 'xc7z020'
INFO: [Common 17-349] Got license for feature 'Implementation' and/or device 'xc7z020'
Running DRC as a precondition to command route_design
INFO: [DRC 23-27] Running DRC with 6 threads
INFO: [Vivado_Tcl 4-198] DRC finished with 0 Errors
INFO: [Vivado_Tcl 4-199] Please refer to the DRC report (report_drc) for more 

INFO: [Timing 38-480] Writing timing data to binary archive.
Writing placer database...
Writing XDEF routing.
Writing XDEF routing logical nets.
Writing XDEF routing special nets.
Write XDEF Complete: Time (s): cpu = 00:00:07 ; elapsed = 00:00:03 . Memory (MB): peak = 3812.176 ; gain = 0.008 ; free physical = 9477 ; free virtual = 46562
INFO: [Common 17-1381] The checkpoint '/mnt/data/thaarres/qkeras_ae_pynq/myproject_vivado_accelerator/project_1.runs/impl_1/design_1_wrapper_routed.dcp' has been generated.
write_checkpoint: Time (s): cpu = 00:00:16 ; elapsed = 00:00:10 . Memory (MB): peak = 3812.176 ; gain = 8.008 ; free physical = 9596 ; free virtual = 46580
INFO: [runtcl-4] Executing : report_drc -file design_1_wrapper_drc_routed.rpt -pb design_1_wrapper_drc_routed.pb -rpx design_1_wrapper_drc_routed.rpx
Command: report_drc -file design_1_wrapper_drc_routed.rpt -pb design_1_wrapper_drc_routed.pb -rpx design_1_wrapper_drc_routed.rpx
INFO: [IP_Flow 19-1839] IP Catalog is up to date.
IN

Loading data files...
Loading site data...
Loading route data...
Processing options...
Creating bitmap...
Creating bitstream...
Writing bitstream ./design_1_wrapper.bit...
INFO: [Vivado 12-1842] Bitgen Completed Successfully.
INFO: [Common 17-83] Releasing license: Implementation
147 Infos, 118 Warnings, 0 Critical Warnings and 0 Errors encountered.
write_bitstream completed successfully
write_bitstream: Time (s): cpu = 00:00:35 ; elapsed = 00:00:28 . Memory (MB): peak = 4184.137 ; gain = 212.957 ; free physical = 9389 ; free virtual = 46392
INFO: [Common 17-206] Exiting Vivado at Thu Dec  2 10:45:48 2021...
[Thu Dec  2 10:45:48 2021] impl_1 finished
wait_on_run: Time (s): cpu = 00:20:10 ; elapsed = 00:12:47 . Memory (MB): peak = 2486.406 ; gain = 0.000 ; free physical = 11581 ; free virtual = 48583
# open_run impl_1
INFO: [Device 21-403] Loading part xc7z020clg400-1
Netlist sorting complete. Time (s): cpu = 00:00:00.38 ; elapsed = 00:00:00.39 . Memory (MB): peak = 2486.406 ; gain = 0.

In [8]:
import os

files = os.lstdir('.')
print(files)

['.git', '.gitignore', 'README.md', 'environment.yml', 'images', 'part1_ADmodel.ipynb', 'part2_hls4ml.ipynb', 'part3_pynqz2.ipynb', 'prepare_adc_data.py', 'util.py', '.ipynb_checkpoints', 'data', 'bkg_dataset.h5', 'Ato4l_dataset.h5', 'baseline_ae.h5', 'compressed_ae.h5', '__pycache__', 'baseline_ae_pynq']
